In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import os
import re

In [20]:
cwd = os.getcwd() 
lists = os.listdir(f'{cwd}/GWAS') 

In [21]:
lists

['DSM субклиническая депрессия logistic.top1000_new.xlsx',
 'DSM_БАР logistic.top1000_new.xlsx',
 'DSM_ГТР logistic.top1000_new.xlsx',
 'DSM_ДЕПРЕССИЯ logistic.top1000_new.xlsx',
 'HADS_A. БАЛЛ linear.top1000_new.xlsx',
 'HADS_A_КАТЕГОРИЯ logistic.top1000_new.xlsx',
 'HADS_D. БАЛЛ linear.top1000_new.xlsx',
 'HADS_D_КАТЕГОРИЯ logistic.top1000_new.xlsx',
 'HADS_D_субклиническая депрессия logistic.top1000_new.xlsx',
 'ангедония logistic.top1000_new.xlsx',
 'гиперсомния logistic.top1000_new.xlsx',
 'гиперсомния плюс гиперфагия logistic.top1000_new.xlsx',
 'гиперфагия logistic.top1000_new.xlsx',
 'любой фенотип logistic.top1000_new.xlsx']

In [25]:
lists = [i.replace('.xlsx', '') for i in lists]

In [27]:
gwas = {}
for i in lists:
    gwas[i] = pd.read_excel(f'GWAS/{i}.xlsx')

In [28]:
for k, v in gwas.items():
    gwas[k]['RSID'] = gwas[k]['RSID'].replace(',chr.+[AGTC]', '', regex = True)
    gwas[k]['RSID'] = gwas[k]['RSID'].replace(';rs\d+', '', regex = True)

# Анализ значимых snp и оценка репликации

In [45]:
def significant_snps(df):
    significant_rs = df[df['P'] < 1e-5]
    return significant_rs

In [46]:
for k, v in gwas.items():
    gwas[k]['RSID'] = gwas[k]['RSID'].replace(',chr.+[AGTC]', '', regex = True)
    significant_rs = significant_snps(gwas[k])
    significant_rs.to_csv(f'Results/0 SignificantRS/SignificantRS_{k}.csv')

In [47]:
def replication_snps(df):
    import pandasgwas as pg
    #significant_rs = df[df['P'] < 1e-5]
    significant_rs = df
    source_id = pg.get_variants_by_efo_id('EFO_0006788').variants
    for i in ['EFO_0005230', 'EFO_0005230', 'EFO_0003756', 'EFO_0007795', 'EFO_0009863', 'EFO_0003761', 'EFO_0009854', 'EFO_0010098', 'EFO_0004262', 'EFO_0004257', 'EFO_0009458', 'EFO_0000677', 'EFO_0003761', 'EFO_0001358', 'EFO_0004247', 'EFO_0007704', 'EFO_0007006', 'MONDO_0004985', 'EFO_0007634', 'EFO_0009963']:
        source_id = pd.concat([source_id, pg.get_variants_by_efo_id(i).variants])
    for i in ['20800221', '34985809', '35879288', '30626913', '33106475', '36750733', '29071344', '27479909', '34924174', '33859377', '25390645', '34634379', '35181757', '33859377']:
        source_id = pd.concat([source_id, pg.get_variants_by_pubmed_id(i).variants])
    replicated_snps = significant_rs[significant_rs['RSID'].isin(source_id['rsId'])]
    gene_replication = significant_rs['GENE_NAME'].value_counts()
    replicated_genes = gene_replication[gene_replication > 1]
    return replicated_snps, replicated_genes

In [48]:
for k, v in gwas.items():
    gwas[k]['RSID'] = gwas[k]['RSID'].replace(',chr.+[AGTC]', '', regex = True)
    replications_rs, replications_genes = replication_snps(gwas[k])
    replications_rs.to_csv(f'Results/1 ReplicatedRS/ReplicatedRS_{k}.csv')
    replications_rs.to_csv(f'Results/1 ReplicatedRS/ReplicatedGENES_{k}.csv')

100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (4 of 4) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (6 of 6) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (13 of 13) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1 of 1) |#########################

# Анализ распределения вероятность причинности

In [50]:
def causal_probability_distributions(df):
    data = df[df['P'] < 1e-5]

    data = data[['RSID', 'BETA', 'SE']]
    data['z'] = data['BETA'] / data['SE']
    data['p'] = 2 * (1 - norm.cdf(abs(data['z'])))
    prior = 0.01  # априорная вероятность
    data['likelihood'] = np.exp(-0.5 * data['z']**2) / (data['SE'] * np.sqrt(2 * np.pi))
    data['posterior'] = data['likelihood'] * prior / (data['likelihood'] * prior + (1 - prior))
    df = df[df['RSID'].isin(data['RSID'])]
    df['Probability'] = data['posterior']
    return df.sort_values(by = 'Probability', ascending=False)

In [51]:
def causal_probability(df):
    data = df[df['P'] < 1e-5]
    
    data = data[['RSID', 'Z_STAT', 'LOG(OR)_SE']]
    data['p'] = 2 * (1 - norm.cdf(abs(data['Z_STAT'])))
    prior = 0.01  # априорная вероятность
    data['likelihood'] = np.exp(-0.5 * data['Z_STAT']**2) / (data['LOG(OR)_SE'] * np.sqrt(2 * np.pi))
    data['posterior'] = data['likelihood'] * prior / (data['likelihood'] * prior + (1 - prior))
    df = df[df['RSID'].isin(data['RSID'])]
    df['Probability'] = data['posterior']
    return df.sort_values(by = 'Probability', ascending=False)

In [54]:
for k, v in gwas.items():
    gwas[k]['RSID'] = gwas[k]['RSID'].replace(',chr.+[AGTC]', '', regex = True)
    try:
        distribution = causal_probability_distributions(gwas[k])
        distribution.to_csv(f'Results/2 DistributionRS/DistributionRS_{k}.csv')
    except:
        distribution = causal_probability(gwas[k])
        distribution.to_csv(f'Results/2 DistributionRS/DistributionRS_{k}.csv')

# Анализ обогащения

In [57]:
def enrich(df):
    significant_rs = df[df['P'] < 1e-5]
    unique_genes = significant_rs['GENE_NAME'].unique()
    unique_genes = [gene for gene in unique_genes if gene != 'intergenic']
    import gseapy as gp

    enr = gp.enrichr(gene_list=unique_genes,
                 organism='human',
                 gene_sets=['GO_Biological_Process_2018','KEGG_2019_Human','WikiPathways_2019_Human','GO_Biological_Process_2017b'],
                 cutoff = 0.5,
                 outdir='enrichr_results')
    return enr.results.sort_values(by='Adjusted P-value')

In [61]:
for k, v in gwas.items():
    gwas[k]['RSID'] = gwas[k]['RSID'].replace(',chr.+[AGTC]', '', regex = True)
    try:
        enrichment = enrich(gwas[k])
        enrichment.to_csv(f'Results/3 Enrichment/Enrichment_{k}.csv')
    except:
        continue


# Pathway analysis

In [63]:
def pathw(df, k):
    significant_rs = df[df['P'] < 1e-5]
    unique_genes = significant_rs['GENE_NAME'].unique()
    unique_genes = [gene for gene in unique_genes if gene != 'intergenic']
    import gseapy as gp
    
    enr_GOBP = gp.enrichr(gene_list=unique_genes,
    gene_sets=['GO_Biological_Process_2021'],
    organism='Human', 
    outdir=f'Results/4 Pathway/GOBP_{k}',
    cutoff=0.5)
    
    enr_GOMF = gp.enrichr(gene_list=unique_genes,
     gene_sets=['GO_Molecular_Function_2021'],
     organism='Human', 
     outdir=f'Results/4 Pathway/GOMF_{k}',
     cutoff=0.5 
     )
    enr_GOCC = gp.enrichr(gene_list=unique_genes,
     gene_sets=['GO_Cellular_Component_2021'],
     organism='Human', 
     outdir=f'Results/4 Pathway/GOCC_{k}',
     cutoff=0.5 
     )
    enr_Reactome = gp.enrichr(gene_list=unique_genes,
     gene_sets=['Reactome_2016'],
     organism='Human', 
     outdir=f'Results/4 Pathway/Reactome_{k}',
     cutoff=0.5 
     )
    


In [64]:
for k, v in gwas.items():
    gwas[k]['RSID'] = gwas[k]['RSID'].replace(',chr.+[AGTC]', '', regex = True)
    try:
        pathway = pathw(gwas[k], k)
    except:
        continue

